In [43]:
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pyarrow.parquet as pq
from types import SimpleNamespace
from pathlib import Path

DATA_DIR = Path("../data")
DATA_FILE = "cb-100-2021001-2021002.parquet"

In [44]:
def bandpass(low_freq, high_freq, sr, data):
  """
  low_freq
  high_freq
  sr - sampling rate
  data - data samples

  Bandpass the signal between the low and high frequency
  """
  # apply a 3-pole bandpass filter
  b, a = scipy.signal.butter(3, [low_freq * 2 /sr, high_freq * 2 / sr], 'band')
  # We use lfilter which is a causal filter that is better suited for onset detection
  # as opposed to filtfilt
  filtered = scipy.signal.lfilter(b, a, data)
  return filtered

In [45]:
table = pq.read_table(DATA_DIR / DATA_FILE)
table.num_rows

80

In [46]:
df = table.to_pandas()
df.head()

,orid,arid,sta,chan,phase,snr,amp,per,delta,mb,ml,ndef,arrival_time,timeres,auto_arrival_time,samprate,calib,data
0,19869965,156900523,MKAR,cb,P,17.811651,0.303406,0.333333,64.235908,3.387758,-999.00000,18,1.609462e+09,-0.259062,1.609462e+09,40.0,1.0,"[0.2685339, 0.27632114, 0.24698758, 0.24447474..."
1,19879955,156901151,MKAR,cb,P,13.424261,0.371238,0.333333,61.445162,3.172225,-999.00000,10,1.609464e+09,0.253651,1.609464e+09,40.0,1.0,"[-0.31641033, -0.31028688, -0.32881105, -0.341..."
2,19879725,156901439,NVAR,cb,P,10.079068,1.098708,0.500000,88.812353,4.253283,4.84218,35,1.609465e+09,0.275838,1.609465e+09,40.0,1.0,"[-0.122175835, -0.12957793, 0.102844924, 0.308..."
3,19879725,156901212,WRA,cb,P,98.458801,23.740281,0.444444,44.015080,4.253283,4.84218,35,1.609464e+09,0.573137,1.609464e+09,40.0,1.0,"[-0.50153637, -0.3003585, -0.103547856, -0.161..."
4,19879725,156901114,ASAR,cb,P,218.175920,9.983312,0.444444,42.964665,4.253283,4.84218,35,1.609464e+09,0.405846,1.609464e+09,20.0,1.0,"[-0.36080712, -0.12755495, -0.1791558, -0.4103..."


In [21]:
np.array(table['data'][0].as_py())

array([ 0.26853389,  0.27632114,  0.24698758, ..., -0.09281293,
       -0.08330281, -0.11970751])

In [33]:
df["arrival_time"].isnull().sum()

0

In [36]:
for idx, row in df.iterrows():
    row = SimpleNamespace(**{name: value for (name, value) in zip(df.columns, row)})
    break
row

namespace(orid=19869965,
          arid=156900523,
          sta='MKAR',
          chan='cb',
          phase='P',
          snr=17.811651,
          amp=0.30340612,
          per=0.33333333,
          delta=64.235908,
          mb=3.3877577,
          ml=-999.0,
          ndef=18,
          arrival_time=1609461832.7622,
          timeres=-0.25906219,
          auto_arrival_time=1609461833.75,
          samprate=40.0,
          calib=1.0,
          data=array([ 0.2685339 ,  0.27632114,  0.24698758, ..., -0.09281293,
                      -0.08330281, -0.11970751], dtype=float32))

In [42]:
num_plot = 0
for idx, row in df.iterrows():
    row = SimpleNamespace(**{name: value for (name, value) in zip(df.columns, row)})
    if row.arrival_time is None or abs(row.arrival_time - row.auto_arrival_time) < 1:
        continue
    
    #xvals = np.arange(0, len(row.data) // row.samprate, 1 / row.samprate)
    filt_data = bandpass(1, 5, row.samprate, row.data)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=filt_data, name="waveform"))
    fig.update_layout(title=f"orid {row.orid} arid {row.arid} sta {row.sta}"
              f" phase {row.phase} chan {row.chan} snr {row.snr} amp {row.amp}",
              xaxis_title="Time (sec)", yaxis_title="Amplitude (nm)")
    fig.add_vline(x=len(row.data)//2, name="Arrival", line_dash="dash", line_color="green")
    #if row.auto_arrival_time is not None:
    #    fig.add_vline(x=row.auto_arrival_time-row.start_time, name="Automatic Arrival", line_dash="dot", line_color="red")
    #fig.add_vline(x=row.arrival_time-row.timeres-row.start_time, name="Theoretical Arrival", line_dash="dashdot", line_color="black", opacity=.5)
    fig.show()

    num_plot += 1
    if num_plot >= 10:
        break
    


In [47]:
df.columns

Index(['orid', 'arid', 'sta', 'chan', 'phase', 'snr', 'amp', 'per', 'delta',
       'mb', 'ml', 'ndef', 'arrival_time', 'timeres', 'auto_arrival_time',
       'samprate', 'calib', 'data'],
      dtype='object')

In [64]:
import hashlib

def is_validation_waveform(waveform_file: str, fold: int, num_folds: int):
    return (
        int(hashlib.md5(waveform_file.encode("utf8")).hexdigest(), 16) % num_folds
    ) == fold

mask = df.apply(lambda row: is_validation_waveform(f"{row.orid}_{row.arid}", 4, 5), axis=1)

In [70]:
df.iloc[0]

orid                                                          19869965
arid                                                         156900523
sta                                                               MKAR
chan                                                                cb
phase                                                                P
snr                                                          17.811651
amp                                                           0.303406
per                                                           0.333333
delta                                                        64.235908
mb                                                            3.387758
ml                                                              -999.0
ndef                                                                18
arrival_time                                           1609461832.7622
timeres                                                      -0.259062
auto_a

In [69]:
df.head()

,orid,arid,sta,chan,phase,snr,amp,per,delta,mb,ml,ndef,arrival_time,timeres,auto_arrival_time,samprate,calib,data
0,19869965,156900523,MKAR,cb,P,17.811651,0.303406,0.333333,64.235908,3.387758,-999.00000,18,1.609462e+09,-0.259062,1.609462e+09,40.0,1.0,"[0.2685339, 0.27632114, 0.24698758, 0.24447474..."
1,19879955,156901151,MKAR,cb,P,13.424261,0.371238,0.333333,61.445162,3.172225,-999.00000,10,1.609464e+09,0.253651,1.609464e+09,40.0,1.0,"[-0.31641033, -0.31028688, -0.32881105, -0.341..."
2,19879725,156901439,NVAR,cb,P,10.079068,1.098708,0.500000,88.812353,4.253283,4.84218,35,1.609465e+09,0.275838,1.609465e+09,40.0,1.0,"[-0.122175835, -0.12957793, 0.102844924, 0.308..."
3,19879725,156901212,WRA,cb,P,98.458801,23.740281,0.444444,44.015080,4.253283,4.84218,35,1.609464e+09,0.573137,1.609464e+09,40.0,1.0,"[-0.50153637, -0.3003585, -0.103547856, -0.161..."
4,19879725,156901114,ASAR,cb,P,218.175920,9.983312,0.444444,42.964665,4.253283,4.84218,35,1.609464e+09,0.405846,1.609464e+09,20.0,1.0,"[-0.36080712, -0.12755495, -0.1791558, -0.4103..."
